In [248]:
# Fix imports when running from notebooks/ folder
import sys
from pathlib import Path

project_root = Path.cwd().parent 
if str(project_root) not in sys.path:
    sys.path.insert(0, str(project_root))

print(f"Added to path: {project_root}")

%load_ext autoreload
%autoreload 2

Added to path: /Users/elshaday/DEV/10Academy/insurance-risk-analysis-week3
The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [ ]:
from src import DataManager
from scripts.constants import Columns
from tabulate import tabulate
import pandas as pd
from scipy.stats import chi2_contingency, ttest_ind
import numpy as np

In [250]:
dm = DataManager()
clean_df = dm.load_csv(load_clean=True)

Basic Data Info:

<class 'pandas.core.frame.DataFrame'>
Index: 846034 entries, 0 to 1000097
Data columns (total 54 columns):
 #   Column                       Non-Null Count   Dtype  
---  ------                       --------------   -----  
 0   UnderwrittenCoverID          846034 non-null  int64  
 1   PolicyID                     846034 non-null  int64  
 2   TransactionMonth             846034 non-null  object 
 3   IsVATRegistered              846034 non-null  bool   
 4   Citizenship                  846034 non-null  object 
 5   LegalType                    846034 non-null  object 
 6   Title                        846034 non-null  object 
 7   Language                     846034 non-null  object 
 8   Bank                         846034 non-null  object 
 9   AccountType                  846034 non-null  object 
 10  MaritalStatus                846034 non-null  object 
 11  Gender                       846034 non-null  object 
 12  Country                      846034 non-null

In [251]:
# KPIs
# Claim Frequency - Claim Occured = 1 if TotalClaims > 0 else 0 ( total number of claims > 0 / total number of policy )
# Claim Severity - Total Claims / Number of Claims for policy with atleast 1 claims ( total amount of claims / total number of claims)
# Margin - TotalPremium - TotalClaims
df = clean_df.copy()

## Null Hypothesis 1: There are no risk differences across provinces

In [252]:
print("H₀:There are no risk differences across provinces (Chi-Square Test):\n")

# Show provinces with sample size
provinces_sorted = (
    df.groupby(Columns.Province.value)
    .agg(
        sample_size=(Columns.PolicyID.value,"count")
    )
    .sort_values(by="sample_size", ascending=False)
    .reset_index()
)
print(provinces_sorted)
print(f"\nTop Province Picks for A/B Testing:\n {provinces_sorted.iloc[0:2,]}")

H₀:There are no risk differences across provinces (Chi-Square Test):

        Province  sample_size
0        Gauteng       338966
1  KwaZulu-Natal       158848
2   Western Cape       137678
3     North West       114906
4     Mpumalanga        33508
5   Eastern Cape        29747
6        Limpopo        20703
7  Northern Cape         6380
8     Free State         5298

Top Province Picks for A/B Testing:
         Province  sample_size
0        Gauteng       338966
1  KwaZulu-Natal       158848


In [253]:
print("Prepare Control and Test Groups from selected Provinces")

group_a = df[df[Columns.Province.value] == 'Gauteng']
group_b = df[df[Columns.Province.value] == "KwaZulu-Natal"]
segmented_df = df[
    (df[Columns.Province.value] == "Gauteng")
    | (df[Columns.Province.value] == "KwaZulu-Natal")
]

print("Group A Info")
print(tabulate(group_a.describe(), headers="keys", tablefmt="grid"))

print("Group B Info")
print(tabulate(group_b.describe(), headers="keys", tablefmt="grid"))

Prepare Control and Test Groups from selected Provinces
Group A Info
+-------+-----------------------+------------+--------------+------------------+--------------------+---------------+-----------------+-------------+-----------------+-----------------------+----------------------+---------------+------------------+----------------------------+------------------+----------------+---------------+
|       |   UnderwrittenCoverID |   PolicyID |   PostalCode |           mmcode |   RegistrationYear |     Cylinders |   cubiccapacity |   kilowatts |   NumberOfDoors |   CustomValueEstimate |   CapitalOutstanding |   CrossBorder |       SumInsured |   CalculatedPremiumPerTerm |   ExcessSelected |   TotalPremium |   TotalClaims |
+=======+=======================+============+==============+==================+====================+===============+=================+=============+=================+=======================+======================+===============+==================+====================

In [259]:
print("H₀:There are no risk differences across provinces (Chi-Square Test):\n")


# Step 1 Group df by policy
grouped_by_policy_df = (
    segmented_df.groupby(Columns.PolicyID.value)
    .agg(
        {
            Columns.TotalPremium.value: "sum",
            Columns.TotalClaims.value: "sum",
            Columns.Province.value: "first",
        }
    )
    .sort_values(by=Columns.TotalPremium.value, ascending=False)
    .reset_index()
)
# Step 2 Classify Policies
grouped_by_policy_df["Claimed"] = np.where(grouped_by_policy_df[Columns.TotalClaims.value] > 0, 1, 0)

# Step 3 Group By Province
grouped_by_province = (
    grouped_by_policy_df.groupby(Columns.Province.value)
    .agg(
        n_claims=("Claimed", "sum"),
        n_policies=(Columns.PolicyID.value, "count"),
        total_claims=(Columns.TotalClaims.value, "sum"),
        total_premiums=(Columns.TotalPremium.value, "sum"),
    )
    .sort_values(by="n_policies", ascending=False)
    .reset_index()
)

# Step 4 Extract KPIs
grouped_by_province["claim_frequency"] = (
    grouped_by_province["n_claims"] / grouped_by_province["n_policies"]
)
grouped_by_province["claim_severity"] = (
    grouped_by_province["total_claims"] / grouped_by_province["n_claims"]
)
grouped_by_province["margin"] = (
    grouped_by_province["total_premiums"] - grouped_by_province["total_claims"]
)
grouped_by_province["loss_ratio"] = (
    grouped_by_province["total_claims"] / grouped_by_province["total_premiums"]
)

print("Aggregated Table:")
print(tabulate(grouped_by_province, headers="keys", tablefmt="grid"))

# Step 5 Chi-Square Test

# Inputs
#   - Observed Frequency (Contingency Table) of the two variables
#   - Expected Frequency
#   - Degree of Freedom (rows - 1) x (columns - 1)
#   - Significance Level - 0.05

print("\nContingency Table:")
cont_table = pd.crosstab(
    grouped_by_policy_df[Columns.Province.value],
    grouped_by_policy_df["Claimed"],
)
print(tabulate(cont_table, headers="keys", tablefmt="grid"))

# Chi-Square Test
chi2, p, dof, expected = chi2_contingency(cont_table)
print(f"\nChi-Square Statistic: {chi2}")
print(f"p-value: {p}")
print(f"Degrees of Freedom: {dof}")

H₀:There are no risk differences across provinces (Chi-Square Test):

Aggregated Table:
+----+---------------+------------+--------------+----------------+------------------+-------------------+------------------+--------------+--------------+
|    | Province      |   n_claims |   n_policies |   total_claims |   total_premiums |   claim_frequency |   claim_severity |       margin |   loss_ratio |
+====+===============+============+==============+================+==================+===================+==================+==============+==============+
|  0 | Gauteng       |        530 |         2302 |    2.58775e+07 |      2.21775e+07 |          0.230235 |          48825.4 | -3.69991e+06 |      1.16683 |
+----+---------------+------------+--------------+----------------+------------------+-------------------+------------------+--------------+--------------+
|  1 | KwaZulu-Natal |        255 |         1315 |    1.41382e+07 |      1.28618e+07 |          0.193916 |          55443.8 | -1.276

### H₀:There are no risk differences across provinces


#### Statistical Hypothesis Testing (Chi-Test)
 - **Question**: Does the distribution of claims occuring differ across provinces?
 - **Answer**: p-value = 0.012
    - p-value is less than significance level of 0.05 which means there statistically IS A significant difference in claims across Provinces
 - **Verdict: Reject Null Hypothesis**  

In [255]:
print("Interpretation and Business Recommendation:\n")


def classify_with_quantiles(series):
    """
    Classify numerical KPI values using quantiles.
    Returns a dict: {value → category}.
    """
    q33 = series.quantile(0.33)
    q66 = series.quantile(0.66)

    def classify(v):
        if v <= q33:
            return "Low"
        elif v <= q66:
            return "Medium"
        else:
            return "High"

    return series.apply(classify)


def compute_risk_score(freq, sev, margin):
    """
    Convert KPI classes into numerical risk scores and compute overall score.
    Higher score = higher risk.
    """
    mapping = {"Low": 0, "Medium": 1, "High": 2}

    return mapping[freq] + mapping[sev] + mapping[margin]


def classify_overall(score):
    """
    Translate total risk score into a final province risk category.
    Score range is 0-6.
    """
    if score >= 4:
        return "High Risk"
    elif score >= 2:
        return "Medium Risk"
    else:
        return "Low Risk"


def classify_kpis(grouped_by_province):
    df = grouped_by_province.copy()

    # 1. Classify each metric using quantiles
    df["frequency_class"] = classify_with_quantiles(df["claim_frequency"])
    df["severity_class"] = classify_with_quantiles(df["claim_severity"])

    # For margin → high risk = LOW margin, so invert using -margin
    df["margin_class"] = classify_with_quantiles(-df["margin"])

    # 2. Compute overall numerical risk score
    df["risk_score"] = df.apply(
        lambda row: compute_risk_score(
            row["frequency_class"], row["severity_class"], row["margin_class"]
        ),
        axis=1,
    )

    # 3. Final classification
    df["Risk Classification"] = df["risk_score"].apply(classify_overall)

    return df

final_risk_df = classify_kpis(grouped_by_province)

print(tabulate(final_risk_df, headers="keys", tablefmt="grid"))

Interpretation and Business Recommendation:

+----+---------------+------------+--------------+----------------+------------------+-------------------+------------------+--------------+--------------+-------------------+------------------+----------------+--------------+-----------------------+
|    | Province      |   n_claims |   n_policies |   total_claims |   total_premiums |   claim_frequency |   claim_severity |       margin |   loss_ratio | frequency_class   | severity_class   | margin_class   |   risk_score | Risk Classification   |
+====+===============+============+==============+================+==================+===================+==================+==============+==============+===================+==================+================+==============+=======================+
|  0 | Gauteng       |        530 |         2302 |    2.58775e+07 |      2.21775e+07 |          0.230235 |          48825.4 | -3.69991e+06 |      1.16683 | High              | Low              | High   

### Breif Interpretation and Business Recommendation

Most provinces exhibit a High Risk classification 
- Gauteng has a highest number of claims (530), highest number of policies (2301), high premiums (2.21699e+07) and high loss ratio (1.167). This province exhibits a negative profit margin and thus puts AlphaCare Insurance Solutions at a High Risk of losing profits. Premiums should be adjusted for this province
- KwaZulu Natal has the second highest number of claims (255), number of policies (1315), high premiums ( 1.28618e+07) and high loss ratio (1.09923). But compared to Gauteng has a lower claim frequency,lower claim severity and profit margin loss. It presents a Moderate Risk of losing profits. Premiums should be adjusted for this province as well



This signifies there is a risk difference among provinces that is not random, and confirms the result of Chi-Square test in Rejecting the Null Hypothesis.



## Null Hypothesis 2: There are no risk differences between zip codes

In [283]:
print("H₀:There are no risk differences between zip codes (Chi-Square Test):")

postal_codes_sorted = (
    df.groupby(Columns.PostalCode.value)
    .agg(n_policies=(Columns.PolicyID.value, "nunique"))
    .reset_index()
    .sort_values("n_policies", ascending=False)
)

top_zips = postal_codes_sorted.head(2)[Columns.PostalCode.value].tolist()
print(f"Top 2 Postal Codes by sample size:\n {top_zips}\n")

postal_group_a = df[df[Columns.PostalCode.value] == top_zips[0]]
postal_group_b = df[df[Columns.PostalCode.value] == top_zips[1]]
postal_segmented_df = df[df[Columns.PostalCode.value].isin(top_zips)]


print("Group A Info:")
print(tabulate(postal_group_a.describe(), headers="keys", tablefmt="grid"))

print("\nGroup B Info:")
print(tabulate(postal_group_b.describe(), headers="keys", tablefmt="grid"))

H₀:There are no risk differences between zip codes (Chi-Square Test):
Top 2 Postal Codes by sample size:
 [2000, 122]

Group A Info:
+-------+-----------------------+------------+--------------+------------------+--------------------+---------------+-----------------+-------------+-----------------+-----------------------+----------------------+---------------+------------------+----------------------------+------------------+----------------+---------------+
|       |   UnderwrittenCoverID |   PolicyID |   PostalCode |           mmcode |   RegistrationYear |     Cylinders |   cubiccapacity |   kilowatts |   NumberOfDoors |   CustomValueEstimate |   CapitalOutstanding |   CrossBorder |       SumInsured |   CalculatedPremiumPerTerm |   ExcessSelected |   TotalPremium |   TotalClaims |
+=======+=======================+============+==============+==================+====================+===============+=================+=============+=================+=======================+==============

In [299]:
print("H₀:There are no risk differences between zip codes (Chi-Square Test):")

# Step 1 Group By Policy ID
postal_grouped_df = (
    postal_segmented_df.groupby(Columns.PolicyID.value)
    .agg(
        {
            Columns.TotalClaims.value: "sum",
            Columns.TotalPremium.value: "sum",
            Columns.PostalCode.value: "first",
        }
    )
    .reset_index()
)

# Step 2 Classify Policies
postal_grouped_df["Claimed"] = np.where(
    postal_grouped_df[Columns.TotalClaims.value] > 0, 1, 0
)

# Step 3 Contingency Table
postal_cont_table = pd.crosstab(
    postal_grouped_df[Columns.PostalCode.value], postal_grouped_df["Claimed"]
)
print("Contingency Table")
print(postal_cont_table)

# Step 4 Chi-Square Test
chi2, p_val, dof, expected = chi2_contingency(postal_cont_table)
print(f"\nChi-Square Statistic: {chi2}")
print(f"p-value: {p}")
print(f"Degrees of Freedom: {dof}")

H₀:There are no risk differences between zip codes (Chi-Square Test):
Contingency Table
Claimed       0    1
PostalCode          
122         174   74
2000        292  144

Chi-Square Statistic: 0.6007287596437055
p-value: 0.43830010628405125
Degrees of Freedom: 1


### H₀:There are no risk differences between zip codes


#### Statistical Hypothesis Testing (Chi-Test)
 - **Question**: Does the distribution of claims occuring differ across zipcodes/postalcodes?
 - **Answer**: p-value = 0.43
    - p-value is greater than significance level of 0.05 which means there statistically IS NO significant difference in claims occuring across zip codes
 - **Verdict: Fail To Reject Hypothesis**  

## Null Hypothesis 3: There is no significant margin (profit) difference between zip codes

In [297]:
print(
    "H₀:There is no significant margin (profit) difference between zip codes (t-test):"
)

# Step 1 Group By Policy ID
postal_grouped_df_for_margin = (
    postal_segmented_df.groupby(Columns.PolicyID.value)
    .agg(
        {
            Columns.TotalClaims.value: "sum",
            Columns.TotalPremium.value: "sum",
            Columns.PostalCode.value: "first",
        }
    )
    .reset_index()
)

# Step 2 Add Margin Per Policy
postal_grouped_df_for_margin["Margin"] = (
    postal_grouped_df_for_margin[Columns.TotalPremium.value]
    - postal_grouped_df_for_margin[Columns.TotalClaims.value]
)

group_a_margins_list = list(postal_grouped_df_for_margin[
    postal_grouped_df_for_margin[Columns.PostalCode.value] == top_zips[0]
]["Margin"])
group_b_margins_list = list(
    postal_grouped_df_for_margin[
        postal_grouped_df_for_margin[Columns.PostalCode.value] == top_zips[1]
    ]["Margin"]
)

t_statistic, p_value = ttest_ind(group_a_margins_list, group_b_margins_list)

print(f"\nT-Test Statistic: {t_statistic}")
print(f"p-value: {p_value}")

H₀:There is no significant margin (profit) difference between zip codes (t-test):

T-Test Statistic: -0.19715382882858534
p-value: 0.8437659014190937


### H₀:There is no significant margin (profit) difference between zip codes


#### Statistical Hypothesis Testing (Chi-Test)
 - **Question**: Does the profit margin differ across zipcodes/postalcodes?
 - **Answer**: p-value = 0.84
    - p-value is greater than significance level of 0.05 which means there statistically IS NO significant difference in profit margins across zip codes
 - **Verdict: Fail To Reject Hypothesis**  

## Null Hypothesis 4: There is no significant risk difference between Women and Men

In [303]:
clean_gender_df = df[df["Gender"] != "Not specified"]

gender_df_group_by_policy = (
    clean_gender_df.groupby(Columns.PolicyID.value)
    .agg(
        {
            Columns.TotalClaims.value: "sum",
            Columns.TotalPremium.value: "sum",
            Columns.Gender.value: "first",
        }
    )
    .reset_index()
)


gender_df_group_by_policy["Claimed"] = np.where(gender_df_group_by_policy["TotalClaims"] > 0, 1,0)

# Contingency Table
gender_cont_table = pd.crosstab(
    gender_df_group_by_policy[Columns.Gender.value],
    gender_df_group_by_policy["Claimed"],
)

print(f"Contingency Table:\n {gender_cont_table}")

# Chi-Square Test
chi2, p, dof, expected = chi2_contingency(gender_cont_table)
print(f"\nChi-Square Statistic: {chi2}")
print(f"p-value: {p}")
print(f"Degrees of Freedom: {dof}")

Contingency Table:
 Claimed   0   1
Gender         
Female   11   6
Male     96  33

Chi-Square Statistic: 0.3126957011662384
p-value: 0.576030687273205
Degrees of Freedom: 1


### H₀:There is no significant risk difference between Women and Men


#### Statistical Hypothesis Testing (Chi-Test)
 - **Question**: Does the risk differ across Genders?
 - **Answer**: p-value = 0.5
    - p-value is greater than significance level of 0.05 which means there statistically IS NO significant difference in risk across genders
 - **Verdict: Fail To Reject Hypothesis**  